# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import csv

# Import API key
from api_keys import g_key

 ### Store WeatherPy results into DataFrame

In [2]:
# Import weatherpy csv file
cities_df = pd.read_csv("city_data.csv")
cities_df

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Sri Aman,MY,1.2376,111.4621,77.09,94,98,2.30
1,1,Kabalo,CD,-6.0500,26.9167,77.67,32,16,3.96
2,2,Cape Town,ZA,-33.9258,18.4232,57.11,88,75,12.66
3,3,Rikitea,PF,-23.1203,-134.9692,74.07,79,99,15.64
4,4,Berlevåg,NO,70.8578,29.0864,51.60,81,20,11.50
...,...,...,...,...,...,...,...,...,...
557,557,Zhezkazgan,KZ,47.8043,67.7144,63.30,62,22,9.17
558,558,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,52.16,72,70,7.52
559,559,Salinópolis,BR,-0.6136,-47.3561,79.36,82,3,11.52
560,560,Tegul'det,RU,57.3064,88.1667,54.50,95,49,3.67


 ### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key) 

In [4]:
humidity_df = cities_df[['City', 'Latitude','Longitude', 'Humidity']]
humidity_df

,City,Latitude,Longitude,Humidity
0,Sri Aman,1.2376,111.4621,94
1,Kabalo,-6.0500,26.9167,32
2,Cape Town,-33.9258,18.4232,88
3,Rikitea,-23.1203,-134.9692,79
4,Berlevåg,70.8578,29.0864,81
...,...,...,...,...
557,Zhezkazgan,47.8043,67.7144,62
558,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,72
559,Salinópolis,-0.6136,-47.3561,82
560,Tegul'det,57.3064,88.1667,95


In [5]:
# Store 'Latitude' and 'Longitude' into  locations 
locations = humidity_df[["Latitude", "Longitude"]].astype(float)

# Convert Humidity % to float and store
humidity = humidity_df["Humidity"].astype(float)

In [6]:
# Create Heatmap layer
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity= np.max(humidity),
                                 point_radius = 3)

fig = gmaps.figure(center=(25.0, -9.0), zoom_level=2)
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [7]:
clean_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & (cities_df["Max Temperature"] >= 70) & (cities_df["Max Temperature"] <= 80)].dropna()

clean_df

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
113,113,Gaza,PS,31.5000,34.4667,73.89,71,0,4.63
115,115,Borūjerd,IR,33.8973,48.7516,77.05,40,0,3.33
137,137,Assiniboia,CA,49.6334,-105.9845,76.82,22,0,9.22
165,165,Evanston,US,42.0411,-87.6901,79.56,63,0,8.05
249,249,Alpena,US,45.0617,-83.4327,71.92,58,0,9.22
340,340,Huesca,ES,42.1362,-0.4087,77.68,47,0,4.61
360,360,Ürümqi,CN,43.8010,87.6005,77.95,44,0,2.24
453,453,Ergani,TR,38.2653,39.7621,76.68,53,0,2.77
460,460,Mujiayingzi,CN,42.1167,118.7833,71.28,62,0,7.00
501,501,Caravelas,BR,-17.7125,-39.2481,70.97,80,0,5.86


### Hotel Map

In [8]:
# Add a "Hotel Name" column to the DataFrame
hotel_df = clean_df.insert(9,'Hotel Name','null')

In [9]:
# Create Hotel dataframe
hotel_df = clean_df[['City','Country','Latitude','Longitude','Hotel Name']]

In [10]:
# Reset index
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Gaza,PS,31.5000,34.4667,null
1,Borūjerd,IR,33.8973,48.7516,null
2,Assiniboia,CA,49.6334,-105.9845,null
3,Evanston,US,42.0411,-87.6901,null
4,Alpena,US,45.0617,-83.4327,null
5,Huesca,ES,42.1362,-0.4087,null
6,Ürümqi,CN,43.8010,87.6005,null
7,Ergani,TR,38.2653,39.7621,null
8,Mujiayingzi,CN,42.1167,118.7833,null
9,Caravelas,BR,-17.7125,-39.2481,null


In [11]:
#Save config information
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {"keyword":"Hotel","type":"Hotel","radius":5000,"key":g_key}

print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create variables
times = 0
response = []

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to parameters dictionary
    parameters["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, parameters).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel availale... skipping.")
        
print("---------------------------")
print("End of Search")

Beginning Data Retrieval     
-----------------------------
Retrieving Results for Index 0: Gaza.
Closest hotel in Gaza is Al Mathaf Hotel.
Retrieving Results for Index 1: Borūjerd.
No hotel availale... skipping.
Retrieving Results for Index 2: Assiniboia.
Closest hotel in Assiniboia is Canalta Hotel Assiniboia.
Retrieving Results for Index 3: Evanston.
Closest hotel in Evanston is Hampton Inn Chicago North-Loyola Station.
Retrieving Results for Index 4: Alpena.
Closest hotel in Alpena is Holiday Inn Express & Suites Alpena - Downtown, an IHG Hotel.
Retrieving Results for Index 5: Huesca.
Closest hotel in Huesca is Hotel Pedro I de Aragón.
Retrieving Results for Index 6: Ürümqi.
Closest hotel in Ürümqi is Sheraton Urumqi Hotel.
Retrieving Results for Index 7: Ergani.
Closest hotel in Ergani is HİLAR OTEL.
Retrieving Results for Index 8: Mujiayingzi.
No hotel availale... skipping.
Retrieving Results for Index 9: Caravelas.
Closest hotel in Caravelas is Pousada Liberdade.
Retrieving Resu

In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Gaza,PS,31.5000,34.4667,Al Mathaf Hotel
1,Borūjerd,IR,33.8973,48.7516,null
2,Assiniboia,CA,49.6334,-105.9845,Canalta Hotel Assiniboia
3,Evanston,US,42.0411,-87.6901,Hampton Inn Chicago North-Loyola Station
4,Alpena,US,45.0617,-83.4327,Holiday Inn Express & Suites Alpena - Downtown...
5,Huesca,ES,42.1362,-0.4087,Hotel Pedro I de Aragón
6,Ürümqi,CN,43.8010,87.6005,Sheraton Urumqi Hotel
7,Ergani,TR,38.2653,39.7621,HİLAR OTEL
8,Mujiayingzi,CN,42.1167,118.7833,null
9,Caravelas,BR,-17.7125,-39.2481,Pousada Liberdade


In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(locations,fill_color='rgba(0, 100, 0, 0.9)', stroke_color='rgba(0, 100, 0, 0.9)', scale=7, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(marker_layer)

In [14]:
# Create a combined map
fig = gmaps.figure(center=(25.0, -9.0), zoom_level=2)

fig.add_layer(humidity_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))